In [145]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from typing import List, Optional
import glob  

plt.rcParams.update({
    'font.family': 'Helvetica',
    'font.size': 26,
    'axes.linewidth': 1.2,
    'axes.labelsize': 26,
    'axes.titlesize': 26,
    'xtick.labelsize': 30,
    'ytick.labelsize': 26,
    'legend.fontsize': 34,
    'figure.dpi': 600,
    'savefig.dpi': 600,
    'savefig.bbox': 'tight',
    'savefig.pad_inches': 0.1,
    'grid.alpha': 0.6,
    'grid.linewidth': 0.8,
    'lines.linewidth': 1.5,
    'patch.linewidth': 0.8
})

In [146]:
def plot_performance(data, output_path: Optional[str] = None, figsize: tuple = (16, 10)):
    """
    Create performance comparison plot for poster presentation.
    
    Parameters:
    -----------
    output_path : str, optional
        Path to save the plot
    figsize : tuple
        Figure size (width, height) in inches
    """
    
    # Strategy names and data
    strategies = ['Column-wise', 'Row-wise', 'Random', 'ACO', 'GA']
    latency_column_wise = data[0] 
    latency_row_wise = data[1]
    latency_random = data[2]
    latency_ACO = data[3]
    latency_GA = data[4]
    
    # Latency data (total_simulation_cycles, computation_occupied_cycles, packet_occupied_cycles, overlap_cycles)
    data_dict = {
        'Column-wise': latency_column_wise,
        'Row-wise': latency_row_wise, 
        'Random': latency_random,
        'ACO': latency_ACO,
        'GA': latency_GA
    }
    
    # Normalize to column-wise baseline (overall latency)
    baseline_total = latency_column_wise[0]  # Column-wise total latency for normalization
    
    # Extract and normalize components for each strategy
    normalized_computation = []
    normalized_data_flow = []
    
    for strategy in strategies:
        strategy_data = data_dict[strategy]
        # Normalize computation and data flow to column-wise total latency baseline
        norm_comp = strategy_data[1] / baseline_total
        norm_data = strategy_data[2] / baseline_total
        
        normalized_computation.append(norm_comp)
        normalized_data_flow.append(norm_data)
    
    # Create the plot
    fig, ax = plt.subplots(figsize=figsize)
    
    # Set up bar positions
    x = np.arange(len(strategies))
    bar_width = 0.6
    
    # Colors for poster visibility
    colors = {
        'computation': "#EF5454",     # Bright red
        'data_flow': "#3077F3",       # Bright blue
    }
    
    # Create stacked bars
    p1 = ax.bar(x, normalized_computation, bar_width,
                label='Computations',
                color=colors['computation'], alpha=0.9,
                edgecolor='black', linewidth=2)
    
    p2 = ax.bar(x, normalized_data_flow, bar_width,
                bottom=normalized_computation,
                label='Data Flow',
                color=colors['data_flow'], alpha=0.9,
                edgecolor='black', linewidth=2)
    
    # Customize for poster presentation
    #ax.set_xlabel('Task Mapping Strategy', fontweight='bold', fontsize=24)
    ax.set_ylabel('Normalized Execution Time')
    #ax.set_title('Performance Comparison of Task Mapping Strategies', fontweight='bold', fontsize=28, pad=20)
    
    # Set x-axis
    ax.set_xticks(x)
    ax.set_xticklabels(strategies, fontweight='bold')
    
    # Y-axis formatting for readability
    ax.tick_params(axis='y')
    
    # Add horizontal line at y=1 for reference (column-wise baseline)
    #ax.axhline(y=1, color='gray', linestyle='--', linewidth=2, alpha=0.7, label='Column-wise baseline')
    
    # Legend with larger font
    ax.legend(loc='upper right', frameon=True, fancybox=True, shadow=True, framealpha=0.9)
    
    # Grid for better readability
    ax.grid(True, axis='y', alpha=0.3, linestyle='--', linewidth=1)
    ax.set_axisbelow(True)
    
    # Make all spines thicker for poster
    for spine in ax.spines.values():
        spine.set_linewidth(2)
    
    # Adjust layout
    plt.tight_layout()
    
    if output_path:
        plt.savefig(output_path, format='svg', bbox_inches='tight', 
                    facecolor='white', edgecolor='none')
        print(f"plot saved to: {output_path}")
    
    plt.show()
    
    # Print summary table
    print("\n" + "="*80)
    print("NORMALIZED PERFORMANCE COMPARISON (Column-wise total latency = 1.0)")
    print("="*80)
    print(f"{'Strategy':<15} {'Norm. Computation':<18} {'Norm. Data Flow':<15} {'Total Normalized':<15}")
    print("-" * 80)
    
    for i, strategy in enumerate(strategies):
        strategy_data = data_dict[strategy]
        total_norm = strategy_data[0] / baseline_total  # Actual total normalized
        norm_comp = normalized_computation[i]
        norm_data = normalized_data_flow[i]
        print(f"{strategy:<15} {norm_comp:<18.3f} {norm_data:<15.3f} {total_norm:<15.3f}")

In [147]:
# Define the file pattern and columns to read
file_pattern = "../data/22Sept_data/model_resnet_early_blocks_12x12_torus_mixed_random_{}.csv"
target_columns = [2, 3, 4, 5, 6]  # computation_occupied_cycles, packet_occupied_cycles, overlap_cycles, total_active_cycles

# Initialize list to store data from all files
all_data = []

# Read files 1-50
for i in range(1, 51):
    file_path = file_pattern.format(i)
    try:
        df = pd.read_csv(file_path)
        # Get the target columns (using iloc for 0-based indexing)
        selected_data = df.iloc[:, target_columns].values
        all_data.append(selected_data)
        #print(f"Read {file_path}: shape {selected_data.shape}")
    except FileNotFoundError:
        print(f"File not found: {file_path}")
    except Exception as e:
        print(f"Error reading {file_path}: {e}")

if all_data:
    # Stack all data and calculate average
    stacked_data = np.stack(all_data, axis=0)
    averaged_data = np.mean(stacked_data, axis=0)

    print(f"\nAveraged data shape: {averaged_data.shape}")
    print("Column names: computation_occupied_cycles, packet_occupied_cycles, overlap_cycles, total_active_cycles")
    print("Averaged values:")
    print(averaged_data)

    # Convert to list format
    averaged_list = averaged_data.tolist()
    print(f"\nAs list: {averaged_list}")
    
    # Save the averaged data for comparison
    latency_random = averaged_list[0]  # First row (assuming single row data)

else:
    print("No data was successfully read")


Averaged data shape: (1, 5)
Column names: computation_occupied_cycles, packet_occupied_cycles, overlap_cycles, total_active_cycles
Averaged values:
[[268555.4  267690.3   11274.94  10412.84 268552.4 ]]

As list: [[268555.4, 267690.3, 11274.94, 10412.84, 268552.4]]


In [148]:
#data to compare: 
#latency, comp, data flow, overlapping, overall_computing time
#latency_row_wise = [170462,168310,8122,5973,170459]
latency_row_wise = [387403,387403,14922,15298,387403]
latency_column_wise = [407403,406776,15922,15298,407400]
latency_random = averaged_data[0].astype(int).tolist()  # Convert to list format
latency_ACO = [169378,167103,5906,3634,169375]
latency_GA = [169214,167339,6633,4761,169211]
data = [latency_column_wise, latency_row_wise, latency_random, latency_ACO, latency_GA]

In [149]:
plot_performance(data, output_path='../data/22Sept_data/plot.svg', figsize=(22, 8))

plot saved to: ../data/22Sept_data/plot.svg



NORMALIZED PERFORMANCE COMPARISON (Column-wise total latency = 1.0)
Strategy        Norm. Computation  Norm. Data Flow Total Normalized
--------------------------------------------------------------------------------
Column-wise     0.998              0.039           1.000          
Row-wise        0.951              0.037           0.951          
Random          0.657              0.028           0.659          
ACO             0.410              0.014           0.416          
GA              0.411              0.016           0.415          
